In [1]:
from z3 import *
from itertools import chain, repeat, combinations, product
from toolz import unique
from operator import itemgetter
from collections import Counter

In [2]:
from z3_utils_hakank import *

In [3]:
data = [line.rstrip(')\n').split(' (contains ')  for line in open('input2020_21.txt', 'r').readlines()]

In [4]:
food_names = list(unique(chain.from_iterable([i.split(' ') for i, j in data])))
allergen_names = list(unique(chain.from_iterable([j.split(', ') for i, j in data])))

In [6]:
allerg = {k:v for v, k in enumerate(allergen_names)}
foods = {k:v for v, k in enumerate(food_names)}

In [13]:
sol = Solver()

In [14]:
# variables
p = {}
n = len(allergen_names)
m = len(food_names)
for i in range(n):
    for j in range(m):
        p[(i,j)] = makeIntVar(sol,"p[%i,%i]" % (i,j),0,1)
#print("p:",p)

In [15]:
# [mxmxvkd, kfcds, sqjhc, nhms, trh, fvjkl, sbzzf]
# [dairy, fish, soy]

In [16]:
# max 1 allergen per food
for j in range(m):
    sol.add(Sum([p[(i,j)] for i in range(n)]) <= 1)

In [17]:
# all allergens must be placed and only at one food
for i in range(n):
    sol.add(Sum([p[(i,j)] for j in range(m)]) == 1)

In [19]:
for i, j in data:
    #A = i.split(' ')
    #B = j.split(', ')
    food_v = list(map(foods.get, i.split(' '))) #itemgetter(*A)(foods)
    allerg_v = list(map(allerg.get, j.split(', '))) #itemgetter(*B)(allerg)
    
    for k in allerg_v:
        l = list(map(lambda x: (k, x), food_v))
        sol.add(And(Or([p[(c, d)] == 1 for c, d in l]))) # product(food_v, allerg_v)

In [20]:
solution = []
sol.check()
mod = sol.model()
for i in range(n):
    for j in range(m):
        if mod.eval(p[(i,j)]) == 1:
            f_sol = list(foods.keys())[list(foods.values()).index(j)]
            a_sol = list(allerg.keys())[list(allerg.values()).index(i)]
            solution.append((f_sol, a_sol))

In [27]:
with_allergen = set(map(lambda x: x[0], solution))
without_allergen = list(set(food_names) - with_allergen)
freq = list(chain.from_iterable([i.split(' ') for i, j in data]))
sum(list(map(Counter(freq).get, without_allergen)))

2307

In [28]:
# Part b

In [29]:
','.join(list(map(lambda x: x[0], sorted(solution, key = lambda x: x[1]))))

'cljf,frtfg,vvfjj,qmrps,hvnkk,qnvx,cpxmpc,qsjszn'